In [47]:
import pandas as pd
from mlx_lm import load, generate
import time
import random

In [5]:
model, tokenizer = load("/Users/austinlackey/Documents/GitHub/llm-data-validation/Finetuned-LLama3-NPSv4")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [123]:
def parse_output(response):
    response = response.split("###")[0]
    if response[-1] == "\n":
        response = response[:-1]
    return response
def generate_output(input: str = None) -> str:
    prompt = "### INSTRUCTION:\nYour task is to review some comments written by National Park data collectors to describe the visitation at their park for a given month. Analyze the comments and identify any months where specific locations were reported as closed. Your response should be 1-2 concise sentences. If no closures are mentioned, please respond with '<Month Name>: No Closures.'. Do not mention locations that are marked as using averages or estimates. Do not include closures that only happened for a few days or temporary closures due to weather or holidays. The format should be as follows:\n<Month Name>: <1-2 concise sentence summary of closures>.\n\n### INPUT:\n"
    prompt = prompt + input + "\n\n### OUTPUT:\n"
    print("PROMPT:\n", prompt)
    response = generate(model, tokenizer, prompt=prompt, verbose=False, temp=0, max_tokens=150)
    print("RESPONSE:\n", response)
    return parse_output(response)
def showcase_output(input: str = None, autocomplete: bool = False) -> str:
    print("INPUT:\n", input)
    output = generate_output(input)
    print()
    print("OUTPUT:\n")
    if autocomplete:
        for word in output.split():
            print(word, end=" ")
            # generate random delay value between 0.01 and 0.3, from a poison distribution
            delay = random.expovariate(10)
            time.sleep(delay)
    return output

In [137]:
data = pd.read_csv("Data/comments_with_prompts.csv")
# data = data.sort_values(by=["Date", "UnitCode"], ascending=[True, True]).reset_index(drop=True)
display(data)

,UnitCode,ParkName,Date,Comments,Year,MonthName,MonthNumber,Prompt,WordCount,TokenCount,CharacterCount
0,RICH,Richmond NBP,2020-09-01,COVID,2020,September,9,Comment for September:\nCOVID,4,5,28
1,DRTO,Dry Tortugas NP,2023-06-01,.,2023,June,6,Comment for June:\n.,4,5,19
2,GOGA,Golden Gate NRA,2020-08-01,COVID,2020,August,8,Comment for August:\nCOVID,4,5,25
3,LYBA,LBJ Memorial Grove on the Potomac,2012-08-01,Estimated,2012,August,8,Comment for August:\nEstimated,4,5,29
4,GOGA,Golden Gate NRA,2020-09-01,COVID,2020,September,9,Comment for September:\nCOVID,4,5,28
...,...,...,...,...,...,...,...,...,...,...,...
34857,GRSM,Great Smoky Mountains NP,2023-08-01,"Abrams Creek - Counter malfunctioning, used 20...",2023,August,8,Comment for August:\nAbrams Creek - Counter ma...,312,478,1924
34858,SARA,Saratoga NHP,2022-10-01,2022 October – Explanatory Unusual factors: 1...,2022,October,10,Comment for October:\n2022 October – Explanato...,299,482,1924
34859,SARA,Saratoga NHP,2022-09-01,2022 September – Explanatory Unusual factors:...,2022,September,9,Comment for September:\n2022 September – Expla...,322,498,2019
34860,WORI,Women's Rights NHP,2004-07-01,"July, 2005 was an exceedingly hot month, with ...",2004,July,7,"Comment for July:\nJuly, 2005 was an exceeding...",294,500,1830


In [143]:
data["Prompt"][len(data)-200]

'Comment for September:\nUnusual factors: 1. Non-automotive counter installed on Tour Road, has yet to be calibrated and statistical formula created to account for pedestrian and bicyclist use. 2. South Gate counter installed, needs to be calibrated, estimate is used.  3. A few days early on had very high heat indices, causing lower visitation. 4. Flooding and washouts from 7/16/2023 still has an adjoining road to Phillips Road —Lohnes Road—closed, reducing traffic load on Phillips. 5. Traffic counter for Route 32 working approx. 2 weeks, so estimate used is double the recorded number.  General: 1. Tour Road cyclists & pedestrians = 3,103; Pedestrians usually slightly higher than cyclists, estimating 55% [1,658] and 45% [1,356] respectively. 2. Schuyler House path estimate used (at 1% of Wilkinson Trail number). 3. South Gate pedestrian count estimate used (at 10% of Wilkinson Trail number). 4. Horse Trail use estimated, but slightly low due to one very hot weekend. 5. No category for 

In [144]:
generate_output(data["Prompt"][len(data)-200])

PROMPT:
 ### INSTRUCTION:
Your task is to review some comments written by National Park data collectors to describe the visitation at their park for a given month. Analyze the comments and identify any months where specific locations were reported as closed. Your response should be 1-2 concise sentences. If no closures are mentioned, please respond with '<Month Name>: No Closures.'. Do not mention locations that are marked as using averages or estimates. Do not include closures that only happened for a few days or temporary closures due to weather or holidays. The format should be as follows:
<Month Name>: <1-2 concise sentence summary of closures>.

### INPUT:
Comment for September:
Unusual factors: 1. Non-automotive counter installed on Tour Road, has yet to be calibrated and statistical formula created to account for pedestrian and bicyclist use. 2. South Gate counter installed, needs to be calibrated, estimate is used.  3. A few days early on had very high heat indices, causing low

'September: No Closures.'

In [57]:
showcase_output(data["Prompt"][len(data)-8], autocomplete=True)

INPUT:
 Comment for March:
Back Country camping opened 2/28/2024.  Need Camp is open for day use only.  Whiskey Creek Group Picnic Area remains closed. Work is continued in the area after the CARR Fire.  March 1-23, the Visitor Center hours were open from 0830-1600, closed Tuesdays and Wednesdays. Starting March 24th, the Visitor Center hours changed to open daily from 0900-1700.  Contract crews are working in the Oak Bottom Complex to build a new ranger station.  Crystal Creek Primitive, Brandy Creek Primitive, and Sheep Camp Primitive are closed for the winter.  All traffic counters were function tested for accuracy.

OUTPUT:

March: Back Country camping opened 2/28/2024. Need Camp is open for day use only. Whiskey Creek Group Picnic Area remains closed. Work is continued in the area after the CARR Fire. March 1-23, the Visitor Center hours were open from 0830-1600, closed Tuesdays and Wednesdays. Starting March 24th, the Visitor Center hours changed to open daily from 0900-1700. Con

'March: Back Country camping opened 2/28/2024.  Need Camp is open for day use only.  Whiskey Creek Group Picnic Area remains closed. Work is continued in the area after the CARR Fire.  March 1-23, the Visitor Center hours were open from 0830-1600, closed Tuesdays and Wednesdays. Starting March 24th, the Visitor Center hours changed to open daily from 0900-1700.  Contract'